# Mass-Spring-Damper System: Generating Training and Test Data

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import time


### Defining WORKSPACE_PATH

# WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
WORKSPACE_PATH = os.path.join(os.getcwd(), '../../../../../')
ROMNet_fld     = os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/')



### Importing External Libraries

import numpy                             as np
import pandas                            as pd


### Importing Matplotlib and Its Style

import matplotlib.pyplot                 as plt

#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/presentation.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/zoomed.mplstyle'))
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_2columns.mplstyle'))


from scipy.integrate import solve_ivp
import pyDOE
# from PCAfold         import PCA          as PCAA

3.9.12 (main, Mar 26 2022, 15:51:13) 
[Clang 12.0.0 (clang-1200.0.32.29)]


Missing colon in file PosixPath('/Users/sventur/.matplotlib/stylelib/paper_2columns.mplstyle'), line 1 ('f#### MATPLOTLIBRC FORMAT')


## Defining Input Variables

In [2]:
OutputDir   = os.path.join(WORKSPACE_PATH, 'ROMNet/Data/Centipede_24Vars_T/')
FigDir      = os.path.join(WORKSPACE_PATH, '../Desktop/Paper_Figures_DeepONet_TEMP/')

n_input     = 24
n_output    = 26

test_perc     = 0.05
valid_perc    = 0.25

ColorVec    = ['#190707', '#dd3232', '#0065a9', '#348a00','#985396','#f68b69']

try:
    os.makedirs(OutputDir)
except OSError as e:
    pass

In [4]:
TempData           = pd.read_csv(OutputDir+'/DATA/temperatures.dat', delim_whitespace=True, header=None)

InputData          = pd.read_csv(OutputDir+'/DATA/input24.dat', delim_whitespace=True, header=None)
NData              = len(InputData)
InputData          = np.repeat(InputData.to_numpy(), 26, axis=0)
InputData          = pd.DataFrame(InputData)
InputData.columns  = input_names
InputData['T']     = np.reshape(np.repeat(TempData.to_numpy().T, NData, axis=0).T, -1)

OutputData         = pd.read_csv(OutputDir+'/DATA/output24.dat', delim_whitespace=True, header=None)
OutputData         = OutputData.to_numpy().T.reshape(-1)[...,np.newaxis]
OutputData         = pd.DataFrame(OutputData)
OutputData.columns = output_names

In [8]:
#hist = InputData.hist(bins=50)

from sklearn import preprocessing

scaler     = preprocessing.StandardScaler().fit(InputData)
input_mean = scaler.mean_
input_std  = scaler.scale_
input_moments = pd.DataFrame(np.concatenate([input_mean[np.newaxis,...], input_std[np.newaxis,...]], axis=0), columns=input_names+['T'])
input_moments.to_csv(OutputDir+'/DATA/Input_Moments.csv', index=False)

InputData_scaled = pd.DataFrame(scaler.transform(InputData), columns = input_names+['T'])



scaler      = preprocessing.StandardScaler().fit(OutputData)
output_mean = scaler.mean_
output_std  = scaler.scale_
output_moments = pd.DataFrame(np.concatenate([output_mean[np.newaxis,...], output_std[np.newaxis,...]], axis=0), columns=output_names)
output_moments.to_csv(OutputDir+'/DATA/Output_Moments.csv', index=False)


OutputData_scaled = pd.DataFrame(scaler.transform(OutputData), columns = output_names)

In [9]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/valid/')
except:
    pass
try:
    os.makedirs(OutputDir+'/test/')
except:
    pass

In [11]:
data_id = 'pts'

Data_ics                     = InputData
n_points_ics                 = len(Data_ics)

idx_ics                      = np.arange(n_points_ics)
other_idx, test_idx          = train_test_split(idx_ics,   test_size=test_perc, random_state=17)
train_idx, valid_idx         = train_test_split(other_idx, test_size=valid_perc, random_state=41)

n_valid                      = len(valid_idx)
n_train                      = len(train_idx)

try:
    os.makedirs(OutputDir+'/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/valid/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/test/'+data_id+'/')
except:
    pass

DataInput  = InputData_scaled[input_names+['T']]
DataInput.iloc[train_idx].to_csv(OutputDir+'/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/valid/'+data_id+'/Input.csv', index=False)
DataInput.iloc[test_idx].to_csv(OutputDir+'/test/'+data_id+'/Input.csv', index=False)

DataOutput = OutputData_scaled[output_names]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/valid/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[test_idx].to_csv(OutputDir+'/test/'+data_id+'/Output.csv', index=False)